# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
import holoviews as hv
from holoviews import opts

hv.extension('bokeh')

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
# Get the current working directory
current_directory = os.getcwd()

# Data file path relative to the current directory
file_name = "cities.csv"
file_path = os.path.join(current_directory, file_name)

# Read El Paso data
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mont-dore,-22.2833,166.5833,27.78,99,50,4.15,NC,1714533205
1,1,enewetak,11.3474,162.3373,27.59,75,34,8.60,MH,1714533205
2,2,port-aux-francais,-49.3500,70.2167,3.40,98,100,19.88,TF,1714533205
3,3,lihue,21.9789,-159.3672,26.58,73,0,3.60,US,1714533205
4,4,aykhal,66.0000,111.5000,-5.97,74,2,4.83,RU,1714533205
...,...,...,...,...,...,...,...,...,...,...
537,537,qamdo,31.1667,97.2333,7.68,37,86,3.39,CN,1714533267
538,538,vargashi,55.3556,65.8467,7.28,47,77,5.54,RU,1714533267
539,539,chiredzi,-21.0500,31.6667,14.54,50,0,1.44,ZW,1714533267
540,540,kota kinabalu,5.9749,116.0724,32.99,75,20,1.54,MY,1714533029


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
humidity = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', 
                                  color='Humidity', cmap='blues', 
                                  hover_cols=['City', 'Humidity'], 
                                  tiles='OSM', frame_width=500, frame_height=400,
                                  title='City Humidity Levels')

# Display the map
humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 50) & (city_data_df["Max Temp"] > 25)]
city_data_df = city_data_df.loc[(city_data_df["Humidity"] > 70)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mont-dore,-22.2833,166.5833,27.78,99,50,4.15,NC,1714533205
1,1,enewetak,11.3474,162.3373,27.59,75,34,8.60,MH,1714533205
3,3,lihue,21.9789,-159.3672,26.58,73,0,3.60,US,1714533205
5,5,west island,-12.1568,96.8225,27.99,89,75,8.23,CC,1714533205
19,19,san luis de since,9.2439,-75.1468,26.48,85,13,4.50,CO,1714533207
...,...,...,...,...,...,...,...,...,...,...
525,525,la'ie,21.6477,-157.9253,27.66,80,39,1.79,US,1714533266
528,528,eydhafushi,5.1033,73.0708,28.40,78,100,4.42,MV,1714533266
529,529,parnamirim,-5.9156,-35.2628,25.19,93,100,2.16,BR,1714533266
532,532,tual,-5.6667,132.7500,28.03,80,100,5.46,ID,1714533267


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mont-dore,NC,-22.2833,166.5833,99,
1,enewetak,MH,11.3474,162.3373,75,
3,lihue,US,21.9789,-159.3672,73,
5,west island,CC,-12.1568,96.8225,89,
19,san luis de since,CO,9.2439,-75.1468,85,
...,...,...,...,...,...,...
525,la'ie,US,21.6477,-157.9253,80,
528,eydhafushi,MV,5.1033,73.0708,78,
529,parnamirim,BR,-5.9156,-35.2628,93,
532,tual,ID,-5.6667,132.7500,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude = row["Lat"]
    Longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mont-dore - nearest hotel: Les Cases de Plum
enewetak - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
west island - nearest hotel: Cocos Village Bungalows
san luis de since - nearest hotel: No hotel found
flying fish cove - nearest hotel: Christmas Island Lodge
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
hawaiian paradise park - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
bubaque - nearest hotel: Chez Julio
mahibadhoo - nearest hotel: No hotel found
bonthe - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
morant bay - nearest hotel: Morant Villas
kailua-kona - nearest hotel: Kona Seaside Hotel
sola - nearest hotel: No hotel found
papao - nearest hotel: Hiti Moana Villa Lodge
georgetown - nearest hotel: Page 63 hostel
kapa'a - nearest hotel: Pono Kai Resort
kampung sungai ara - nearest hotel: Olive Tree Hotel
utrik - nearest hotel: No hotel found
saipan - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mont-dore,NC,-22.2833,166.5833,99,Les Cases de Plum
1,enewetak,MH,11.3474,162.3373,75,No hotel found
3,lihue,US,21.9789,-159.3672,73,Kauai Palms
5,west island,CC,-12.1568,96.8225,89,Cocos Village Bungalows
19,san luis de since,CO,9.2439,-75.1468,85,No hotel found
...,...,...,...,...,...,...
525,la'ie,US,21.6477,-157.9253,80,No hotel found
528,eydhafushi,MV,5.1033,73.0708,78,The Westin Maldives Miriandhoo Resort
529,parnamirim,BR,-5.9156,-35.2628,93,Pousada Santana
532,tual,ID,-5.6667,132.7500,80,Suita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
humidity = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                    color='Humidity', cmap='blues', 
                                    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],
                                    tiles='OSM', frame_width=500, frame_height=400, 
                                    title='City Humidity Levels'
)

# Display the map
humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)